<a href="https://www.kaggle.com/code/laansdole/stt-rag-tts-project?scriptVersionId=241797907" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# STT

In [1]:
!pip install SpeechRecognition

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 50.4 MB/s eta 0:00:00


In [2]:
from pydub import AudioSegment
import os

def convert_mp3_to_wav(mp3_file_path, wav_file_path):
    audio = AudioSegment.from_mp3(mp3_file_path)
    audio.export(wav_file_path, format="wav")

directory_path = "/kaggle/input/mini-speech-diarization/dataset/raw"
wav_directory_path = "/kaggle/working/converted_wav_files"

os.makedirs(wav_directory_path, exist_ok=True)

for filename in sorted(os.listdir(directory_path)):
    if filename.endswith(".mp3"):
        mp3_file_path = os.path.join(directory_path, filename)
        wav_file_path = os.path.join(wav_directory_path, filename.replace(".mp3", ".wav"))
        convert_mp3_to_wav(mp3_file_path, wav_file_path)
        print(f"Converted {mp3_file_path} to {wav_file_path}")


Converted /kaggle/input/mini-speech-diarization/dataset/raw/TOEFL Listening Practice Test.mp3 to /kaggle/working/converted_wav_files/TOEFL Listening Practice Test.wav


In [3]:
import speech_recognition as sr
import os
import pandas as pd

def speech_to_text_from_audio(audio_file_path, language="en-US", chunk_length=60000):
    recognizer = sr.Recognizer()
    
    audio = AudioSegment.from_wav(audio_file_path)
    total_duration = len(audio)
    texts = []

    for i in range(0, total_duration, chunk_length):
        chunk = audio[i:i + chunk_length]
        chunk.export("temp.wav", format="wav")
        with sr.AudioFile("temp.wav") as source:
            audio_chunk = recognizer.record(source)
        
        try:
            text = recognizer.recognize_google(audio_chunk, language=language)
            texts.append(text)
        except sr.UnknownValueError:
            print(f"Google Speech Recognition could not understand audio from {audio_file_path} at {i} ms")
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")

    return " ".join(texts)

directory_path = "/kaggle/working/converted_wav_files"

text = []
for filename in sorted(os.listdir(directory_path)):
    if filename.endswith(".wav"):
        audio_file_path = os.path.join(directory_path, filename)
        text.append(speech_to_text_from_audio(audio_file_path))

In [4]:
text

["hi I'm Lucas I'm the TOEFL expert at magoosh.com and in this video you're going to get some listening practice that's just like listening on the real TOEFL but first let me introduce you to the listening section so you know what to expect you're going to see or hear three recordings this is a full set on the TOEFL listening but on a real TOEFL you'll have two sets with six recordings altogether period rewind and listen again just listen once and take notes because after every recording you're going to see questions five or six multiple choice questions meaning A B C or D write your answer as the question is on the screen similar to the real test you'll have about 30 seconds per question so we will keep that question on the screen in the video for about 30 seconds and in that time you should write down your answer your notes from what you heard before and then you'll see the next question at the end of all three recordings with all of their questions you'll see the answers so you can 

# RAG

In [5]:
!pip install langchain

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 438.3/438.3 kB 26.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.3/360.3 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
  Attempting uninstall: requests-toolbelt
    Found existing installation: requests-toolbelt 0.10.1
    Uninstalling requests-toolbelt-0.10.1:
      Successfully uninstalled requests-toolbelt-0.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 24.8.2 requires cubinlinker, which is not installed.
cudf 24.8.2 

In [6]:
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00


In [7]:
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.4/63.4 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 19.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 46.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 352.5/352.5 kB 19.6 MB/s eta 0:00:00


In [8]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 6.1 MB/s eta 0:00:00


In [9]:
!pip install faiss-gpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 20.3 MB/s eta 0:00:00


In [10]:
!pip install -U langsmith

In [11]:
import bs4
from langchain import hub
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough, RunnableSequence
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.docstore.document import Document
from langchain.embeddings import HuggingFaceEmbeddings
from sentence_transformers import SentenceTransformer
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.chains.question_answering import load_qa_chain
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from transformers import RobertaForCausalLM, RobertaTokenizer, pipeline

In [12]:
doc = Document(page_content=text[0])
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)

splits = text_splitter.split_documents([doc])
len(splits)

17

In [13]:
splits

[Document(metadata={}, page_content="hi I'm Lucas I'm the TOEFL expert at magoosh.com and in this video you're going to get some listening practice that's just like listening on the real TOEFL but first let me introduce you to the listening section so you know what to expect you're going to see or hear three recordings this is a full set on the TOEFL listening but on a real TOEFL you'll have two sets with six recordings altogether period rewind and listen again just listen once and take notes because after every recording you're going to see questions five or six multiple choice questions meaning A B C or D write your answer as the question is on the screen similar to the real test you'll have about 30 seconds per question so we will keep that question on the screen in the video for about 30 seconds and in that time you should write down your answer your notes from what you heard before and then you'll see the next question at the end of all three recordings with all of their questions

In [14]:
from transformers import RagTokenizer, RagRetriever, RagTokenForGeneration, pipeline
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch 

embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents=splits, embedding=embedding_model)

retriever = vectorstore.as_retriever()

tokenizer = AutoTokenizer.from_pretrained("/kaggle/input/gemma-2/transformers/gemma-2-2b-it/1/")
model = AutoModelForCausalLM.from_pretrained(
    "/kaggle/input/gemma-2/transformers/gemma-2-2b-it/1/",
    device_map="auto",
)


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")  
model.to(device) 

def generate_answer(question):
    related_docs = retriever.get_relevant_documents(question)
    context = " ".join([doc.page_content for doc in related_docs])
    input_text = f"{context}\n\nQuestion: {question}\nAnswer:"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to(device)  
    generated_ids = model.generate(input_ids, max_length=1024, num_return_sequences=1)
    return tokenizer.decode(generated_ids[0], skip_special_tokens=True)

/tmp/ipykernel_23/2722621526.py:7: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [15]:
question = "What is this document about?"
answer = generate_answer(question)
print(answer)

/tmp/ipykernel_23/2722621526.py:23: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  related_docs = retriever.get_relevant_documents(question)


you've looked at the study guide I assumed kind of yes I mean I haven't gone through the whole thing yet I'm still just doing the stuff on the first page you haven't looked at the second page of the study guide at all yet know I've barely had time to go back over the first two readings from the first page well no wonder you're so confused about where to start let me show you a copy of the study guide look here see what it says at the top of the page this first page is just a list of all the texts and videos that'll be on the test the actual guide is on the second page see oh wow and it says what I need to remember from eating so for example from the intro to adolescent psychology book you only need to review the developmental stages from chapter 2 and it looks like I need to know everything from the main textbook the big one right you want to review all the Core Concepts in the first four chapters of development through lifespan well almost all I forgot to mention this on the study to 

In [16]:
def extract_answer(text):
    if '\nAnswer:' in text:
        answer = text.split('\nAnswer:')[1].strip()
    else:
        answer = "Answer 부분을 찾을 수 없습니다."
    
    return answer

only_answer = extract_answer(answer)
print(only_answer)

This document is about the study guide for a psychology test. 

**Explanation:**

The document is a conversation between a student and a professor about the psychology test. The student is confused about where to start studying and the professor provides guidance on the study guide. 

Let me know if you have any other questions.


# TTS

In [17]:
!pip install gtts

/opt/conda/lib/python3.10/pty.py:89: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  pid, fd = os.forkpty()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [18]:
from gtts import gTTS
import os

def text_to_speech(text, language='en', output_file='output.mp3'):
    """

    Parameters:
    text (str): 변환할 텍스트
    language (str): 음성 언어 (기본값은 영어 'en')
    output_file (str): 저장할 mp3 파일 이름 (기본값은 'output.mp3')
    """
    tts = gTTS(text=text, lang=language, slow=False)

    tts.save(output_file)
    print(f"음성 파일이 {output_file}로 저장되었습니다.")

    os.system(f"start {output_file}")  # Windows에서 작동
    # os.system(f"afplay {output_file}")  # macOS에서 작동
    # os.system(f"mpg321 {output_file}")  # Linux에서 작동

text_to_speech(only_answer, language='en', output_file='hello.mp3')


음성 파일이 hello.mp3로 저장되었습니다.


sh: 1: start: not found
